In [ ]:
!pip install tdmclient

In [34]:
# pip install tdmclient
from tdmclient import ClientAsync, aw
import numpy as np

import importlib, Local_Nav
import Local_Nav
importlib.reload(Local_Nav)

from Local_Nav import control_step, PathTracker, PATH_TOL
# 1) Build your path
wps = [(0,0), (300,0), (300,200), (0,200)]
tracker = PathTracker(wps, path_tol=PATH_TOL, waypoint_tol=50.0)

# 2) Pose struct
class Pose: pass
pose = Pose()
pose.x, pose.y, pose.theta = 0.0, 0.0, 0.0  # initialize from your odometry


def main():
    client = ClientAsync()
    node = aw(client.wait_for_node())     # finds the wireless Thymio via TDM
    try:
        aw(node.lock())                       # lock the node for R/W
    except Exception:
        pass # ignore it it wasn't locked
    
        # Stop any running Aseba program so it doesn’t override your motor commands
    aw(node.lock())
    aw(node.stop())

    print("Connected:", node)
    try:
        # Read sensors and drive for a few seconds
        for k in range(100):  # ~5 s at 20 Hz
            aw(node.wait_for_variables({"prox.horizontal"}))
            prox = list(node["prox.horizontal"])       # length 7
        

            # actual avoidance
            ml, mr, st = control_step(pose, prox, tracker)
            aw(node.set_variables({
                "motor.left.target":  [ml],
                "motor.right.target": [mr],
            }))

            print(f"prox={prox} ml={ml} mr={mr}")
    finally:
        # Stop and unlock at the end
        aw(node.set_variables({
            "motor.left.target":  [0],
            "motor.right.target": [0],
        }))
        aw(node.unlock())

if __name__ == "__main__":
    main() 

Connected: Node 30f587b4-fb96-452d-aa4f-57791877b058
going back to path
prox=[0, 0, 1445, 1218, 0, 0, 0] ml=80 mr=80
going back to path
prox=[0, 0, 1445, 1218, 0, 0, 0] ml=80 mr=80
going back to path
prox=[0, 0, 1465, 1265, 0, 0, 0] ml=80 mr=80
going back to path
prox=[0, 0, 1465, 1265, 0, 0, 0] ml=80 mr=80
going back to path
prox=[0, 0, 1465, 1265, 0, 0, 0] ml=80 mr=80
FW side=-1 front=[0, 0, 1535, 1437, 0] side_sensor=1437 e_dist=1063 w=-53.2
prox=[0, 0, 1535, 1437, 0, 0, 0] ml=101 mr=-5
FW side=-1 front=[0, 0, 1785, 1722, 0] side_sensor=1722 e_dist=778 w=-38.9
prox=[0, 0, 1785, 1722, 0, 0, 0] ml=86 mr=9
FW side=-1 front=[0, 0, 1785, 1722, 0] side_sensor=1722 e_dist=778 w=-38.9
prox=[0, 0, 1785, 1722, 0, 0, 0] ml=86 mr=9
FW side=-1 front=[0, 0, 1978, 1756, 0] side_sensor=1756 e_dist=744 w=-37.2
prox=[0, 0, 1978, 1756, 0, 0, 0] ml=85 mr=10
FW side=-1 front=[0, 0, 1978, 1756, 0] side_sensor=1756 e_dist=744 w=-37.2
prox=[0, 0, 1978, 1756, 0, 0, 0] ml=85 mr=10
FW side=-1 front=[0, 1524, 

In [7]:
client = ClientAsync()
node = aw(client.wait_for_node())
# Stop and unlock at the end
aw(node.set_variables({
    "motor.left.target":  [0],
    "motor.right.target": [0],
}))
aw(node.unlock())

{'error_code': 2}

In [ ]:
# Test local nav
from Local_Nav import control_step
# 1) Build your path
wps = [(0,0), (300,0), (300,200), (0,200)]
tracker = PathTracker(wps, path_tol=PATH_TOL, waypoint_tol=50.0)

# 2) Pose struct
class Pose: pass
pose = Pose()
pose.x, pose.y, pose.theta = 0.0, 0.0, 0.0  # initialize from your odometry

# 3) In your control loop (e.g., every 50 ms)
# Read sensors (prox) and pose here
# prox must be a list/array of length 7 from Thymio: 5 front + 2 rear
# pose.x/y/theta from your estimator

ml, mr, st = control_step(pose, prox, tracker)

# Send to Thymio motors (tdmclient example sketch)
# with client.lock():
#     node["motor.left.target"] = ml
#     node["motor.right.target"] = mr